In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
session_name = '20230510'
save_path = f'/home/zhangyuhao/Desktop/Result/Online_Move_Correct/Spectrum_analysis/{session_name}'
evalues = np.load(save_path+'/evalues.npy')
evectors = np.load(save_path+'/evectors.npy')
mags = np.load(save_path+'/mags.npy')
# Slice the 'mags' numpy array along the second dimension
sliced_mags = [mags[:, i, :] for i in range(4)]
stages = ['stage1','stage2','stage3','stage4']
# Convert the 2D ndarrays into dataframes
dfs = []
for i, mag_2d in enumerate(sliced_mags):
    df = pd.DataFrame({
        'mags': mag_2d.flatten(),
        'stage': stages[i]
    })
    dfs.append(df)

# Concatenate all dataframes into a single dataframe
result_df = pd.concat(dfs, ignore_index=True)
# Filter out rows where 'mags' column values are less than 0.02
result_df = result_df[result_df['mags'] >= 0.02]
print(result_df)
result_df = result_df[result_df['mags'] >= 0.02]
print(result_df)


          mags   stage
1     1.713264  stage1
2     0.942627  stage1
3     0.942627  stage1
4     1.118415  stage1
5     1.118415  stage1
...        ...     ...
4085  1.000000  stage4
4086  1.000000  stage4
4109  1.000000  stage4
4338  1.000000  stage4
4348  1.000000  stage4

[373 rows x 2 columns]


In [ ]:

plt.figure(figsize=(10, 6))
sns.swarmplot(
    x='stage', 
    y='mags', 
    data=result_df, 
    palette="viridis",
    size=3          # 调整点的大小
)

plt.title("Swarm Plot of Mags by Stage")
plt.xlabel("Stage")
plt.ylabel("Mags")
plt.show()

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

sns.stripplot(
    x='stage', 
    y='mags', 
    data=result_df, 
    jitter=True,  # 添加抖动防止点重叠
    palette="viridis",
    alpha=0.6      # 半透明效果
)

plt.title("Distribution of Mags Across Stages")
plt.xlabel("Stage")
plt.ylabel("Mags Value")
plt.show()


In [ ]:
#sliced_evectors = [evectors[:, i, :, :] for i in range(4)]
trial_evectors = [evectors[i, :, :, :] for i in range(10)]
for i, sliced in enumerate(trial_evectors):  # each trial
    print(f"Sliced evectors {i+1} shape: {sliced.shape}")  
    sliced_2d_arrays = [sliced[i, :, :] for i in range(4)]

    # 提取每个二维数组的唯一行向量
    unique_vectors = [np.unique(arr, axis=0) for arr in sliced_2d_arrays]

    # 计算四个数组的共有向量
    common_set = set(map(tuple, unique_vectors[0]))
    for uv in unique_vectors[1:]:
        common_set.intersection_update(map(tuple, uv))
    common_vectors = np.array(list(common_set))  # 转换为NumPy数组

    # 计算每个数组的非共有向量（排除共有向量）
    non_common_vectors = []
    for i in range(4):
        # 生成掩码：当前数组的向量不在共有向量中
        mask = ~np.any(np.all(unique_vectors[i][:, None] == common_vectors, axis=2), axis=1)
        non_common_vectors.append(unique_vectors[i][mask])

    # 可视化配置
    plot_config = [
        {'color': 'green', 'marker': 's', 'label': 'Phase 1 Non-Common'},  # 绿色方块
        {'color': 'gold', 'marker': '^', 'label': 'Phase 2 Non-Common'},    # 黄色三角
        {'color': 'red', 'marker': '*', 'label': 'Phase 3 Non-Common'},     # 红色五角星
        {'color': 'blue', 'marker': 'x', 'label': 'Phase 4 Non-Common'},    # 蓝色叉子
    ]

    plt.figure(figsize=(14, 8))
    time_points = np.arange(110)

    # 初始化标签控制变量
    common_plotted = False  # 新增：控制共有向量标签
    phase_plotted = [False] * 4  # 新增：控制各阶段标签

    # 1. 绘制共有向量（黑色）
    for vec in common_vectors:
        plt.plot(time_points, vec, 
                color='black', linewidth=1.5, alpha=0.6, 
                label='Common Vectors' if not common_plotted else None)
        common_plotted = True  # 确保标签只添加一次
    '''
    # 2. 绘制各阶段非共有向量
    phases = ['Phase 1', 'Phase 2', 'Phase 3', 'Phase 4']
    colors = ['green', 'gold', 'red', 'blue']
    markers = ['s', '^', '*', 'x']

    for phase_idx in range(4):
        vectors = non_common_vectors[phase_idx]
        for vec in vectors:
            plt.plot(time_points, vec,
                    color=colors[phase_idx], 
                    marker=markers[phase_idx],
                    markersize=6,
                    markevery=5,
                    linewidth=1.2,
                    alpha=0.7,
                    label=f'{phases[phase_idx]}' if not phase_plotted[phase_idx] else None)
        phase_plotted[phase_idx] = True  # 确保每个阶段标签只添加一次
    '''
    # 3. 图表装饰
    plt.title('Time Series Comparison: Common vs Non-Common Vectors', fontsize=14)
    plt.xlabel('Time Points', fontsize=12)
    plt.ylabel('Value', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)

    # 去重图例
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))  # 自动去重
    plt.legend(by_label.values(), by_label.keys(), loc='upper right')

    plt.tight_layout()
    plt.savefig('time_series_comparison.png', dpi=300)
    plt.show()


In [ ]:
#sliced_evectors = [evectors[:, i, :, :] for i in range(4)]
trial_evectors = [evectors[i, :, :, :] for i in range(10)]
for i, sliced in enumerate(trial_evectors):  # each trial
    print(f"Sliced evectors {i+1} shape: {sliced.shape}")  
    sliced_2d_arrays = [sliced[i, :, :] for i in range(4)]
    # Compare the four 2D ndarrays and count the same elements
    same_elements_count = np.sum(np.all([sliced_2d_arrays[0] == sliced_2d_arrays[1],
                                         sliced_2d_arrays[1] == sliced_2d_arrays[2],
                                         sliced_2d_arrays[2] == sliced_2d_arrays[3]], axis=0))

    # Find the different elements between these four 2D ndarrays
    different_elements = np.logical_or.reduce([sliced_2d_arrays[0] != sliced_2d_arrays[1],
                                               sliced_2d_arrays[1] != sliced_2d_arrays[2],
                                               sliced_2d_arrays[2] != sliced_2d_arrays[3]])

    print(f"Number of same elements: {same_elements_count}")
    #print(f"Different elements:\n{sliced_2d_arrays[0][different_elements]}")
    # Count the number of different elements in each stage
    different_elements_count = [np.sum(different_elements[i, :]) for i in range(4)]

    # Create a bar plot to show the count of different elements in each stage
    plt.figure(figsize=(10, 6))
    plt.bar(stages, different_elements_count, color='skyblue')
    plt.title("Count of Different Elements in Each Stage")
    plt.xlabel("Stage")
    plt.ylabel("Count of Different Elements")
    plt.show()
    #for i, array in enumerate(sliced_2d_arrays):  # each stage
        #print(f"Sliced 2D array {i+1} shape: {array.shape}")